In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
class Borrower:
    """
    Represents an individual borrower who experiences income shocks,
    borrows money, and repays loans based on their financial status.
    """
    def __init__(self, params):
        self.beta = params.get('beta', 0.9)  # Time discount factor
        self.gamma = params.get('gamma', 2.0)  # Risk aversion coefficient
        self.rho = params.get('rho', 0.85)  # Income persistence parameter
        self.sigma = params.get('sigma', 0.15)  # Income volatility
        self.initial_wealth = params.get('initial_wealth', 2.0)  # Initial wealth level
        self.borrowing_limit = params.get('borrowing_limit', 1.0)  # Maximum loan allowed
        
        self.wealth = self.initial_wealth  # Initial wealth
        self.y = 0.0  # Initial income level
        self.consumption = 0.0  # Initial consumption level
        self.credit_history = []  # List to track past income values
        self.previous_loans = []  # Track previous loan repayment behavior
    
    def update_income(self):
        """
        Updates the borrower's income using an autoregressive process.
        """
        epsilon = np.random.normal(0, 1)
        self.y = self.rho * self.y + self.sigma * epsilon
        self.income_history.append(self.income)
        return self.income
        
    def update_wealth(self, interest_rate):
        """
        Updates the borrower's wealth based on consumption, income, and interest rate.
        """
        self.assets = self.wealth - self.consumption
        self.wealth = interest_rate * (self.wealth - self.consumption) + np.exp(self.y)
        return self.wealth
        
    def check_borrowing_constraint(self, consumption):
        """
        Checks if consumption satisfies the borrowing constraint:
            c_t < w_t + borrowing_limit
        """
        return consumption <= self.wealth + self.borrowing_limit    

    def expected_utility(self, consumption_path):
        """
        ∑ β^t u(c_t) = ∑ 0.9^t * (-1/c_t)
        """
        utility_sum = 0
        for t, consumption in enumerate(consumption_path):
            utility_sum += (self.beta ** t) * self.utility(consumption)
        return utility_sum

In [5]:
class Lender:
    """
    Represents a lender that provides loans to borrowers and determines interest rates
    based on borrower creditworthiness.
    """
    def __init__(self, params):
        self.beta_l = params.get('beta_l', 0.95)  # Lender's discount factor
        self.theta = params.get('theta', 0.1)  # Sensitivity of interest rates to credit scores
        self.available_funds = params.get('initial_funds', 100.0)  # Initial capital available for lending
    
    def set_interest_rate(self, borrower):
        """
        Determines the interest rate for a borrower based on their credit score.
        """
        pass
    
    def lend_money(self, borrower):
        """
        Provides a loan to the borrower within the available funds.
        """
        if self.available_funds <= 0:
            return 0.0
        max_loan = min(borrower.borrowing_limit, self.available_funds)
        loan = np.random.uniform(0, max_loan)
        self.available_funds -= loan
        return loan
